In [1]:
import os
import json
import jsonlines
import time
import pandas as pd
from IPython.display import display

from lkae.utils.data_loading import pkl_dir, load_pkls, root_dir, AuredDataset
from lkae.retrieval.retrieve import get_retriever, retrieve_evidence, AuredDataset

datasets = load_pkls(pkl_dir)

# possilbe splits: train, dev, train_dev_combined
# (test, all_combined don't have "labels")
split = 'dev'

dataset_split = f'English_{split}'
qrel_filename = f'{dataset_split}_qrels.txt'

dataset_variations_dict = datasets[dataset_split]
print(dataset_variations_dict.keys())

import pyterrier as pt
import pyterrier.io as ptio
import pyterrier.pipelines as ptpipelines
from ir_measures import R, MAP    

if not pt.started():
    pt.init()

dict_keys(['nopre-nam-bio', 'nopre-nam-nobio', 'nopre-nonam-bio', 'nopre-nonam-nobio', 'pre-nam-bio', 'pre-nam-nobio', 'pre-nonam-bio', 'pre-nonam-nobio'])


PyTerrier 0.10.1 has loaded Terrier 5.10 (built by craigm on 2024-08-22 17:33) and terrier-helper 0.0.8



In [2]:
# ground truth RQ1
golden = ptio.read_qrels(os.path.join(root_dir, 'data', qrel_filename))

# select a set of variations of the dataset
# these selected variations are selected for these reasons:
# - pre-nonam-nobio     ("raw" data, but preprocessed)
# - pre-nam-bio         (we would expect lexical retrieval to be best here)
# - nopre-nonam-nobio   ("raw" data)
# - nopre-nam-bio       (we would expect semantic retrieval to be best here, most information contained here)
selected_variations = ["pre-nonam-nobio", "nopre-nonam-nobio"]

In [3]:
# load each config and construct its retriever

retrievers = {}

with open('config.json', 'r') as file:
    configs = json.load(file)

    for config in configs['configs']:
        retriever_label = get_retriever(**config)
        retrievers[config['retriever_method']] = retriever_label

retrievers

{'bm25': <lkae.retrieval.methods.bm25.BM25Retriever at 0x20b0483e530>,
 'rerank-nv-embed-v1': <lkae.retrieval.methods.rerank_bm25_nv.RerankingRetriever at 0x20b0483f730>}

In [4]:
# then for every variation of the dataset in ds, run the experiment with each retriever and save the results

out_dir = 'results'
data = []

for selected_variation in selected_variations:
    dataset: AuredDataset = dataset_variations_dict[selected_variation]
    for retriever_label in retrievers:
        start = time.time()

        retrieved_data = retrieve_evidence(dataset[:], retrievers[retriever_label])

        pred = pd.DataFrame([[*d, retriever_label] for d in retrieved_data], columns=['qid', 'docno', 'rank', 'score', 'name']) 

        eval = ptpipelines.Evaluate(pred, golden, metrics = [R@5,MAP], perquery=False)
        r5, meanap = [v for v in eval.values()]

        score = r5

        wall_time = time.time() - start

        print(f'result for retrieval run - R@5: {r5:.4f} MAP: {meanap:.4f} with config\tretriever: {retriever_label};\tds: {selected_variation}, took {wall_time:.2f} seconds')
        
        data.append({
            'R5': r5,
            'MAP': meanap,
            'Retrieval_Method': retriever_label, 
            'DS_Settings': selected_variation,
            'Time (s)': wall_time,
        })

# Convert the list of dictionaries to a DataFrame
df_retrieval = pd.DataFrame(data)

df_retrieval.to_csv(f'{out_dir}/df_retrieval.csv')
print(f'saved df to {out_dir}/df_retrieval.csv')

# Display the DataFrame
display(df_retrieval.sort_values(by='R5', ascending=False))

result for retrieval run - R@5: 0.7214 MAP: 0.6454 with config	retriever: bm25;	ds: pre-nonam-nobio, took 0.58 seconds
result for retrieval run - R@5: 0.8242 MAP: 0.7840 with config	retriever: rerank-nv-embed-v1;	ds: pre-nonam-nobio, took 140.10 seconds
result for retrieval run - R@5: 0.6688 MAP: 0.6085 with config	retriever: bm25;	ds: nopre-nonam-nobio, took 0.50 seconds
result for retrieval run - R@5: 0.8242 MAP: 0.8016 with config	retriever: rerank-nv-embed-v1;	ds: nopre-nonam-nobio, took 143.98 seconds
saved df to results/df_retrieval.csv


,R5,MAP,Retrieval_Method,DS_Settings,Time (s)
1,0.824211,0.784023,rerank-nv-embed-v1,pre-nonam-nobio,140.100415
3,0.824211,0.801567,rerank-nv-embed-v1,nopre-nonam-nobio,143.983665
0,0.721404,0.645351,bm25,pre-nonam-nobio,0.584160
2,0.668772,0.608509,bm25,nopre-nonam-nobio,0.495882
